In [7]:
df.head()

,simulant_id_1,simulant_id_2,full_name_1,full_name_2,address_1,address_2,city_1,city_2,dob_1,dob_2,...,race_1,race_2,name_similarity,address_similarity,city_similarity,match,dob_similarity,state_match,sex_match,race_match
0,0_14147,0_6,Clara Amaya,Elan Alonso Tellez,6877 prospect ave,1501 interlake ave n,Anytown,Anytown,2002-05-20,2013-07-30,...,Latino,Latino,28,38,100,NM,0.6,1,0,1
1,0_14147,0_5622,Clara Amaya,Kaylee Castillo,6877 prospect ave,3 stoke ct,Anytown,Anytown,2002-05-20,2003-06-10,...,Latino,Latino,23,37,100,NM,0.7,1,1,1
2,0_14147,0_7254,Clara Amaya,Caleb Rodriguez,6877 prospect ave,927 broomfield ln,Anytown,Anytown,2002-05-20,2007-01-10,...,Latino,White,23,35,100,NM,0.7,1,0,0
3,0_14147,0_7255,Clara Amaya,Henry Rodriguez,6877 prospect ave,927 broomfield ln,Anytown,Anytown,2002-05-20,2007-07-09,...,Latino,White,15,35,100,NM,0.7,1,0,0
4,0_14147,0_7256,Clara Amaya,Kynleigh Rodriguez,6877 prospect ave,927 broomfield ln,Anytown,Anytown,2002-05-20,2017-05-06,...,Latino,White,14,35,100,NM,0.7,1,1,0


In [11]:
X_train

array([[1.0, 0.0, 1.0, ..., 0.9, 29, 17],
       [1.0, 0.0, 0.0, ..., 0.5, 27, 31],
       [0.0, 1.0, 0.0, ..., 0.7, 44, 31],
       ...,
       [0.0, 1.0, 0.0, ..., 0.6, 33, 29],
       [1.0, 0.0, 1.0, ..., 0.6, 36, 29],
       [0.0, 1.0, 1.0, ..., 0.7, 33, 26]], dtype=object)

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from fuzzywuzzy import fuzz  # Make sure you have fuzzywuzzy installed: pip install fuzzywuzzy python-Levenshtein
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression # Example model, you can change this
from sklearn.metrics import classification_report
import numpy as np

# ... (Load your dataframe 'df' here) ...
df = pd.read_parquet('decennial_census_pairs.parquet')

# Define the features (including original ones)
features = ['full_name_1', 'full_name_2', 'address_1', 'address_2', 'race_1', 'race_2', 'state_1', 'state_2', 'dob_1', 'dob_2', 'sex_1', 'sex_2', 'city_similarity']

# Convert dob to string
df['dob_1'] = df['dob_1'].astype(str)
df['dob_2'] = df['dob_2'].astype(str)

# Create match column
df['match'] = np.where(df['simulant_id_1'] == df['simulant_id_2'], 'M', 'NM')

# Extract features and target
X = df[features]
y = df['match']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fuzzy Matching Functions
def calculate_name_similarity(row):
    return fuzz.ratio(row['full_name_1'], row['full_name_2'])

def calculate_address_similarity(row):
    return fuzz.ratio(row['address_1'], row['address_2'])

def calculate_dob_similarity(row):
    return fuzz.ratio(row['dob_1'], row['dob_2']) / 100 # Normalize to 0-1 range

# Apply fuzzy matching to train and test sets
X_train['name_similarity'] = X_train.apply(calculate_name_similarity, axis=1)
X_train['address_similarity'] = X_train.apply(calculate_address_similarity, axis=1)
X_train['dob_similarity'] = X_train.apply(calculate_dob_similarity, axis=1)

X_test['name_similarity'] = X_test.apply(calculate_name_similarity, axis=1)
X_test['address_similarity'] = X_test.apply(calculate_address_similarity, axis=1)
X_test['dob_similarity'] = X_test.apply(calculate_dob_similarity, axis=1)

# Add race, state, and sex match columns
X_train['race_match'] = np.where(X_train['race_1'] == X_train['race_2'], 1, 0)
X_train['state_match'] = np.where(X_train['state_1'] == X_train['state_2'], 1, 0)
X_train['sex_match'] = np.where(X_train['sex_1'] == X_train['sex_2'], 1, 0)

X_test['race_match'] = np.where(X_test['race_1'] == X_test['race_2'], 1, 0)
X_test['state_match'] = np.where(X_test['state_1'] == X_test['state_2'], 1, 0)
X_test['sex_match'] = np.where(X_test['sex_1'] == X_test['sex_2'], 1, 0)

# Drop unnecessary columns
columns_to_drop = ['dob_1', 'dob_2', 'full_name_1', 'full_name_2', 'address_1', 'address_2', 'race_1', 'race_2', 'state_1', 'state_2', 'sex_1', 'sex_2']
X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)

# Convert y_train and y_test to numerical values for Logistic Regression.
y_train = np.where(y_train == 'M', 1, 0)
y_test = np.where(y_test == 'M',1,0)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


# If you need to transform your data back to a pandas dataframe after one-hot encoding:
X_train_df = pd.DataFrame(X_train, columns=preprocessor.get_feature_names_out())
X_test_df = pd.DataFrame(X_test, columns=preprocessor.get_feature_names_out())

# ... (Rest of your code) ...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1830191
           1       1.00      1.00      1.00       366

    accuracy                           1.00   1830557
   macro avg       1.00      1.00      1.00   1830557
weighted avg       1.00      1.00      1.00   1830557



In [16]:
X_train

,city_similarity,name_similarity,address_similarity,dob_similarity,race_match,state_match,sex_match
2819438,100,17,29,0.9,0,1,0
4565382,100,31,27,0.5,0,1,1
4894571,100,31,44,0.7,1,1,1
3383841,100,25,32,0.6,1,1,0
7507594,100,26,36,0.7,0,1,1
...,...,...,...,...,...,...,...
7204212,100,15,28,0.5,0,1,1
2234489,100,31,24,0.6,0,1,1
4304572,100,29,33,0.6,1,1,1
6550634,100,29,36,0.6,0,1,0
